In [1]:
from recom.datasets import load_ml_small_rating

# load data
dataset = load_ml_small_rating(need_raw=True)

# load features
ratings = dataset['raw']
ratings_train_dict = dataset['train_dict']
ratings_test_dict = dataset['test_dict']
n_user = dataset['n_user']
n_item = dataset['n_item']
user2ix = dataset['user2ix']
ix2user = dataset['ix2user']
item2ix = dataset['item2ix']
ix2item = dataset['ix2item']

del dataset

print(f'Users: {n_user}, Items: {n_item}. Sparsity: {round(1-len(ratings)/n_user/n_item, 4)}')
print(f'User reduced from {len(user2ix.keys())} to {len(ratings_train_dict.keys())}')

Users: 610, Items: 9724. Sparsity: 0.983
User reduced from 610 to 607


In [197]:
from torch import nn
from torch import Tensor, LongTensor


class ItemAutoEncoder(nn.Module):
    def __init__(self, n_user, hidden_dim=64
                     , encode_actv_func=nn.Sigmoid
                     , decode_actv_func=nn.Identity
                     , dropout=0.05) -> None:
        super(ItemAutoEncoder, self).__init__()
        # encoder: (n_user, hidden_dim)
        self.encoder = nn.Sequential(
            nn.Linear(n_user, hidden_dim, bias=True)
            , encode_actv_func()
        )
        # decoder: (hidden_dim, n_user)
        self.decoder = nn.Sequential(
            nn.Linear(hidden_dim, n_user, bias=True)
            , decode_actv_func()
        )
        # dropout to reduce overfitting
        self.dropout = nn.Dropout(p=dropout)
        
    def pred(self, input):

        hidden = self.dropout(self.encoder(input))
        ratings = self.decoder(hidden)
        
        return ratings

    def forward(self, input):
        """ Only for training.
            We will use mask in this part to control gradients.
        """
        ratings = self.pred(input)
        # we contrain the output value to zero
        # , which, mathematically, can control the gradient of 
        # non-observed rating-related params to zero.
        mask = (input!=0)

        return ratings * mask

In [3]:
def rating_vectorize(rat_dict, n_user, n_item, view='item'):
    from torch import zeros

    rat_mat = zeros(n_user, n_item)
    for u in rat_dict:
        for i in rat_dict[u]:
            rat_mat[u, i] = rat_dict[u][i]
    
    if view == 'item':
        return rat_mat.T
    else:
        return rat_mat

train_mat = rating_vectorize(ratings_train_dict, n_user=n_user, n_item=n_item)
train_mat

tensor([[0.0000, 0.0000, 0.0000,  ..., 2.5000, 3.0000, 5.0000],
        [0.0000, 0.0000, 0.0000,  ..., 2.0000, 0.0000, 0.0000],
        [4.0000, 0.0000, 0.0000,  ..., 2.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])

In [141]:
def autorec_data_loader(rat_dict, n_user, n_item, view='item'
                        , use_sampling=False, obj_size=512
                        , batch_size=256):
    from torch.utils.data import DataLoader
    from random import choices
    from torch import LongTensor

    rating_mat = rating_vectorize(rat_dict, n_user, n_item, view)

    if use_sampling:
        by_axis = n_item if view=='item' else n_user
        rand_ixs = choices(list(range(by_axis)), obj_size)

        return DataLoader(dataset=rating_mat[LongTensor(rand_ixs)]
                          , batch_size=batch_size
                          , shuffle=True)
    else:
        return DataLoader(dataset=rating_mat, batch_size=batch_size, shuffle=True)

dl = autorec_data_loader(ratings_train_dict, n_user, n_item)

In [206]:
import time
from torch import autograd, LongTensor, device
from torch import optim
import numpy as np

def maskedSE(input:Tensor, target:Tensor):
    mask = (target!=0)

    return ((input[mask]-target[mask])**2)

rating_mat = rating_vectorize(ratings_train_dict, n_user, n_item)
test_mat = rating_vectorize(ratings_test_dict, n_user, n_item)

model = ItemAutoEncoder(n_user, hidden_dim=128)
opt = optim.Adam(model.parameters(), lr=0.0005, weight_decay=1e-4)

USE_CUDA = True
N_EPOCHS = 32
report_interval=1


if USE_CUDA:
    compute_device = device('cuda')
    model.cuda()
    rating_mat = rating_mat.to(compute_device)
    test_mat = test_mat.to(compute_device)
else:
    compute_device = device('cpu')

train_loss_by_ep = []
test_rmse_by_ep = []

t0 = time.time()
for epoch in range(N_EPOCHS):
    train_data = autorec_data_loader(ratings_train_dict, n_user, n_item)

    ep_loss = []
    for i, batch in enumerate(train_data):

        model.zero_grad()

        inputs = autograd.Variable(batch).to(compute_device)

        preds = model(inputs)
        loss = maskedSE(input=preds, target=inputs)
        loss.mean().backward()
        opt.step()
        ep_loss.extend(loss.data.to(compute_device).tolist())

    train_loss_by_ep.append(np.sqrt(np.mean(ep_loss)))
    
    # test
    preds = model.pred(rating_mat)
    test_rmse = maskedSE(preds, test_mat).mean().sqrt()
    test_rmse_by_ep.append(test_rmse.data.to(compute_device).tolist())

    if report_interval > 0 \
            and ((epoch+1) % report_interval == 0):
        
        t1=time.time()
        print(f'Epoch: {epoch+1}, Time: {round(t1-t0,2)}, /Average train loss {round(sum(train_loss_by_ep[-report_interval:])/report_interval, 5)}')
        print(f'\t\t\t/Average test loss {round(sum(test_rmse_by_ep[-report_interval:])/report_interval, 5)}')
        t0=time.time()

model = model.to('cpu')

Epoch: 1, Time: 30.91, /Average train loss 3.03924
			/Average test loss 2.44192
Epoch: 2, Time: 12.67, /Average train loss 1.66055
			/Average test loss 1.24215
Epoch: 3, Time: 1.0, /Average train loss 1.08988
			/Average test loss 0.96048
Epoch: 4, Time: 0.9, /Average train loss 1.01298
			/Average test loss 0.92298
Epoch: 5, Time: 0.92, /Average train loss 0.99674
			/Average test loss 0.91355
Epoch: 6, Time: 0.85, /Average train loss 0.98642
			/Average test loss 0.90529
Epoch: 7, Time: 0.87, /Average train loss 0.9732
			/Average test loss 0.89176
Epoch: 8, Time: 0.85, /Average train loss 0.95789
			/Average test loss 0.88146
Epoch: 9, Time: 0.85, /Average train loss 0.94281
			/Average test loss 0.86179
Epoch: 10, Time: 0.95, /Average train loss 0.92764
			/Average test loss 0.84772
Epoch: 11, Time: 0.85, /Average train loss 0.91337
			/Average test loss 0.83214
Epoch: 12, Time: 0.83, /Average train loss 0.89985
			/Average test loss 0.82036
Epoch: 13, Time: 0.9, /Average train l